# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from warnings import filterwarnings 
filterwarnings(action="ignore")

In [2]:
import os
print(os.getcwd())

c:\Users\ARKO BERA\Desktop\MLOPS\MLOPS_Project_1\notebook


In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

display(train.head())
display(test.head())

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8
1,750001,male,26,200.0,97.0,20.0,101.0,40.5
2,750002,female,29,188.0,85.0,16.0,102.0,40.4
3,750003,female,39,172.0,73.0,20.0,107.0,40.6
4,750004,female,30,173.0,67.0,16.0,94.0,40.5


In [4]:
train.drop(columns=['id'],inplace=True)
test.drop(columns=['id'],inplace=True)

# EDA

In [11]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from warnings import filterwarnings

filterwarnings("ignore")



class Load_Data:
    def __init__(self, file_path=None,file_df=None):
        if file_path is not None:
            self.df = pd.read_csv(file_path)
        elif file_df is not None:
            self.df = file_df
        #self.id = self.df['id']
        #self.df.drop(columns=['id'], inplace=True)
        print("Shape = ", self.df.shape)

    def summarize(self, include='all'):
        print("=" * 50, 'SUMMARY', '=' * 50)
        if include == 'numerical':
            summarize_df = self.df.describe(include=['number']).T
        elif include == 'categorical':
            summarize_df = self.df.describe(include=['object', 'category']).T
        else:
            summarize_df = self.df.describe(include='all').T

        summarize_df['dtype'] = self.df.dtypes
        summarize_df['missing'] = self.df.isnull().sum()
        summarize_df['unique'] = self.df.nunique()
        summarize_df['duplicates'] = self.df.duplicated().sum()
        summarize_df['most_frequent'] = self.df.select_dtypes(include=['object', 'category']).apply(
            lambda col: col.value_counts().idxmax() if col.nunique() > 0 else None
        )

        def highlight(val):
            if isinstance(val, (int, float)):
                if val > 100000:
                    return 'background-color: red'
                elif val > 50000:
                    return 'background-color: orange'
                elif val > 10000:
                    return 'background-color: blue'
                elif val < 1000:
                    return 'background-color: green'
            return ''

        summarize_df.drop(columns=['25%', '50%', '75%', 'count', 'most_frequent'], inplace=True)
        styled_df = summarize_df.style.applymap(highlight, subset=['missing', 'unique'])
        return styled_df

    def visualize(self, include='all', sample=10000, exclude=[], target=None):
        sample_df = self.df.sample(sample)
    
        if include == 'numerical':
            columns_to_plot = self.df.select_dtypes(include=['number']).columns
        elif include == 'categorical':
            columns_to_plot = self.df.select_dtypes(include=['object', 'category']).columns
        else:
            columns_to_plot = self.df.columns
    
        columns_to_plot = [col for col in columns_to_plot if col not in exclude]
    
        if 'numerical' in include or 'all' in include:
            print("=" * 50, 'Visualizing Numerical Features', '=' * 50)
            numerical_cols = self.df.select_dtypes(include=['number']).columns
            for col in numerical_cols:
                if col not in exclude:
                    fig = make_subplots(
                        rows=1, cols=3,
                        subplot_titles=(f'{col} - Histogram', f'{col} - Boxplot', f'{col} vs {target}'),
                        column_widths=[0.33, 0.33, 0.33]
                    )
                
                    hist = px.histogram(sample_df, x=col, title=f'{col} - Histogram')
                    fig.add_trace(hist.data[0], row=1, col=1)
                
                    box = px.box(sample_df, y=col, title=f'{col} - Boxplot')
                    fig.add_trace(box.data[0], row=1, col=2)
                
                    if target and target in self.df.columns:
                        box_target = px.box(sample_df, x=target, y=col, title=f'{col} vs {target}')
                        fig.add_trace(box_target.data[0], row=1, col=3)
                
                    fig.update_layout(title=f'{col} - Distribution and Feature Dependence', showlegend=False)
                    fig.show()
    
        if 'categorical' in include or 'all' in include:
            print("=" * 50, 'Visualizing Categorical Features', '=' * 50)
            categorical_cols = self.df.select_dtypes(include=['object', 'category']).columns
            for col in categorical_cols:
                if col not in exclude:
                    fig = make_subplots(
                        rows=1, cols=3,
                        subplot_titles=(f'{col} - Count Plot', f'{col} - Pie Chart', f'{col} vs {target}'),
                        column_widths=[0.33, 0.33, 0.33],
                        specs=[[{"type": "bar"}, {"type": "pie"}, {"type": "bar"}]]
                    )
                
                    count_data = sample_df[col].value_counts().reset_index()
                    count_data.columns = [col, 'count']
                    count = px.bar(count_data, x=col, y='count', title=f'{col} - Count Plot')
                    fig.add_trace(count.data[0], row=1, col=1)
                
                    pie = px.pie(sample_df, names=col, title=f'{col} - Pie Chart')
                    fig.add_trace(pie.data[0], row=1, col=2)
                
                    if target and target in self.df.columns:
                        category_mean = sample_df.groupby(col)[target].mean().reset_index()
                        cat_target = px.bar(category_mean, x=col, y=target, title=f'{col} vs {target}')
                        fig.add_trace(cat_target.data[0], row=1, col=3)
                
                    fig.update_layout(title=f'{col} - Distribution and Feature Dependence', showlegend=True)
                    fig.show()


    def impute_columns(self, strategies=None, constant_values=None):
        if strategies is None:
            print("No strategies provided. Automatically imputing missing values.")
            strategies = {}

            for col in self.df.columns:
                if self.df[col].dtype in ['int64', 'float64']:
                    strategies[col] = 'mean'
                else:
                    strategies[col] = 'mode'

        for col, strategy in strategies.items():
            if col not in self.df.columns:
                print(f"Warning: Column '{col}' not found in the dataframe. Skipping.")
                continue

            if strategy == 'mean':
                if self.df[col].dtype in ['int64', 'float64']:
                    self.df[col].fillna(self.df[col].mean(), inplace=True)
                    print(f"Imputed '{col}' with mean.")
                else:
                    print(f"Skipping '{col}' as it's not numeric for mean imputation.")

            elif strategy == 'median':
                if self.df[col].dtype in ['int64', 'float64']:
                    self.df[col].fillna(self.df[col].median(), inplace=True)
                    print(f"Imputed '{col}' with median.")
                else:
                    print(f"Skipping '{col}' as it's not numeric for median imputation.")

            elif strategy == 'mode':
                if not self.df[col].isnull().all():
                    self.df[col].fillna(self.df[col].mode().iloc[0], inplace=True)
                    print(f"Imputed '{col}' with mode.")
                else:
                    print(f"Cannot compute mode for '{col}' as all values are NaN.")

            elif strategy == 'constant':
                if constant_values and col in constant_values:
                    self.df[col].fillna(constant_values[col], inplace=True)
                    print(f"Imputed '{col}' with constant value '{constant_values[col]}'.")
                else:
                    print(f"Error: Provide a constant value for column '{col}' in 'constant_values' dictionary.")

            else:
                print(f"Error: Unsupported strategy '{strategy}' for column '{col}'. Use 'mean', 'median', 'mode', or 'constant'.")

    def feature_target_dependence(self, target_col, exclude=[]):
        """
        Analyze the dependence of the target column on other features.

        Parameters:
        - target_col (str): The target column name.
        - exclude (list): List of feature column names to exclude from analysis.

        Returns:
        - pd.DataFrame: Summary of the dependence analysis.
        """
        if target_col not in self.df.columns:
            raise ValueError(f"Target column '{target_col}' not found in the dataset.")
        
        dependence_summary = []

        for col in self.df.columns:
            if col == target_col or col in exclude:
                continue
            
            # Numerical target
            if pd.api.types.is_numeric_dtype(self.df[target_col]):
                if pd.api.types.is_numeric_dtype(self.df[col]):
                    # Drop rows with NaN in either column
                    valid_data = self.df[[col, target_col]].dropna()
                    if len(valid_data) > 1:
                        stat, p_value = stats.pearsonr(valid_data[col], valid_data[target_col])
                        dependence_summary.append([col, 'numerical', 'Pearson Correlation', stat, p_value])
                    else:
                        dependence_summary.append([col, 'numerical', 'Pearson Correlation', 'Insufficient Data', 'N/A'])
                elif pd.api.types.is_categorical_dtype(self.df[col]) or pd.api.types.is_object_dtype(self.df[col]):
                    # Perform ANOVA
                    valid_data = self.df[[col, target_col]].dropna()
                    if len(valid_data[col].unique()) > 1 and len(valid_data) > 1:
                        groups = [valid_data[valid_data[col] == level][target_col] for level in valid_data[col].unique()]
                        stat, p_value = stats.f_oneway(*groups)
                        dependence_summary.append([col, 'categorical', 'ANOVA', stat, p_value])
                    else:
                        dependence_summary.append([col, 'categorical', 'ANOVA', 'Insufficient Data', 'N/A'])

            # Categorical target
            elif pd.api.types.is_categorical_dtype(self.df[target_col]) or pd.api.types.is_object_dtype(self.df[target_col]):
                if pd.api.types.is_categorical_dtype(self.df[col]) or pd.api.types.is_object_dtype(self.df[col]):
                    # Perform Chi-Square Test
                    contingency_table = pd.crosstab(self.df[col], self.df[target_col])
                    if contingency_table.size > 1:
                        stat, p_value, _, _ = stats.chi2_contingency(contingency_table)
                        dependence_summary.append([col, 'categorical', 'Chi-Square Test', stat, p_value])
                    else:
                        dependence_summary.append([col, 'categorical', 'Chi-Square Test', 'Insufficient Data', 'N/A'])
                elif pd.api.types.is_numeric_dtype(self.df[col]):
                    # Perform ANOVA
                    valid_data = self.df[[col, target_col]].dropna()
                    if len(valid_data[target_col].unique()) > 1 and len(valid_data) > 1:
                        groups = [valid_data[valid_data[target_col] == level][col] for level in valid_data[target_col].unique()]
                        stat, p_value = stats.f_oneway(*groups)
                        dependence_summary.append([col, 'numerical', 'ANOVA', stat, p_value])
                    else:
                        dependence_summary.append([col, 'numerical', 'ANOVA', 'Insufficient Data', 'N/A'])

        df =  pd.DataFrame(dependence_summary, columns=['Feature', 'Feature Type', 'Test Used', 'Statistic', 'p-value'])
        df['p-value'] = pd.to_numeric(df['p-value'], errors='coerce')  # Convert to numeric, set errors to NaN if not possible
        df['p-value'] = df['p-value'].apply(lambda x: round(x, 5) if pd.notnull(x) else x)  # Round only if not NaN
        #df['p-value'] = df['p-value'].apply(lambda x:round(x,5))
        def highlight(val):
            if isinstance(val, (int, float)):
                if val <0.05:
                    return 'background-color:green'     
            return ''

        dep_df = df.style.applymap(highlight, subset=['p-value'])
        return dep_df
        
    def get_df(self):
        return self.df

In [12]:
train_loader = Load_Data(file_df=train)
test_loader = Load_Data(file_df=test)

Shape =  (750000, 8)
Shape =  (250000, 7)


In [13]:
train_loader.summarize(include='all')

================================================== SUMMARY ==================================================


,unique,top,freq,mean,std,min,max,dtype,missing,duplicates
Sex,2,female,375721,nan,nan,nan,nan,object,0,2841
Age,60,nan,nan,41.420404,15.175049,20.000000,79.000000,int64,0,2841
Height,86,nan,nan,174.697685,12.824496,126.000000,222.000000,float64,0,2841
Weight,91,nan,nan,75.145668,13.982704,36.000000,132.000000,float64,0,2841
Duration,30,nan,nan,15.421015,8.354095,1.000000,30.000000,float64,0,2841
Heart_Rate,63,nan,nan,95.483995,9.449845,67.000000,128.000000,float64,0,2841
Body_Temp,75,nan,nan,40.036253,0.779875,37.100000,41.500000,float64,0,2841
Calories,277,nan,nan,88.282781,62.395349,1.000000,314.000000,float64,0,2841


In [14]:
train_loader.visualize(target='Calories')

================================================== Visualizing Numerical Features ==================================================


================================================== Visualizing Categorical Features ==================================================


In [15]:
train_loader.feature_target_dependence(target_col="Calories")

,Feature,Feature Type,Test Used,Statistic,p-value
0,Sex,categorical,ANOVA,108.221994,0.000000
1,Age,numerical,Pearson Correlation,0.145683,0.000000
2,Height,numerical,Pearson Correlation,-0.004026,0.000490
3,Weight,numerical,Pearson Correlation,0.015863,0.000000
4,Duration,numerical,Pearson Correlation,0.959908,0.000000
5,Heart_Rate,numerical,Pearson Correlation,0.908748,0.000000
6,Body_Temp,numerical,Pearson Correlation,0.828671,0.000000


# Encoding

In [16]:
train['Sex'] = train['Sex'].map({'female':0,"male":1})
test['Sex'] = test['Sex'].map({'female':0,"male":1})

# Modelling

In [17]:
import xgboost
from sklearn.model_selection import KFold

In [19]:
class Trainer:
    def __init__(self,X,y,folds,test):

        self.X = X
        self.y = np.log1p(y)
        self.folds = folds
        self.test = test
        self.oofs = np.zeros(len(self.X))
        self.pred = np.zeros(len(self.test))
        
    def train(self):
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        for fold,(train_indx,test_indx) in enumerate(kf.split(self.X,self.y)):
            print(f"Fold : {fold+1}")
            x_train, y_train = self.X.iloc[train_indx],self.y.iloc[train_indx]
            x_valid, y_valid = self.X.iloc[test_indx], self.y.iloc[test_indx]

            model = xgboost.XGBRegressor(
                device='cuda',
                max_depth=8,
                colsample_bytree=0.9,
                subsample=0.9,
                n_estimators=1500,
                learning_rate=0.007,
                random_state=0,
                eval_metric="rmse"
            )
            model.fit(
                x_train, y_train,
                eval_set=[(x_valid, y_valid)],  
                verbose=300,
            )
            self.oofs[test_indx] = model.predict(x_valid)
            self.pred += model.predict(test)

In [18]:
X = train.copy()
y = X.pop('Calories')

In [ ]:
%%time
trainer = Trainer(X,y,5,test)
trainer.train()

Fold : 1
[0]	validation_0-rmse:0.95589
[300]	validation_0-rmse:0.13756
[600]	validation_0-rmse:0.06378
[900]	validation_0-rmse:0.06041
[1200]	validation_0-rmse:0.06004
[1499]	validation_0-rmse:0.05992
Fold : 2
[0]	validation_0-rmse:0.95833
[300]	validation_0-rmse:0.13844
[600]	validation_0-rmse:0.06445
[900]	validation_0-rmse:0.06080
[1200]	validation_0-rmse:0.06035
[1499]	validation_0-rmse:0.06022
Fold : 3
[0]	validation_0-rmse:0.95695
[300]	validation_0-rmse:0.13746
[600]	validation_0-rmse:0.06347
[900]	validation_0-rmse:0.06003
[1200]	validation_0-rmse:0.05965
[1499]	validation_0-rmse:0.05955
Fold : 4
[0]	validation_0-rmse:0.95545
[300]	validation_0-rmse:0.13715
[600]	validation_0-rmse:0.06372
[900]	validation_0-rmse:0.06040
[1200]	validation_0-rmse:0.06002
[1499]	validation_0-rmse:0.05993
Fold : 5
[0]	validation_0-rmse:0.95620
[300]	validation_0-rmse:0.13748
[600]	validation_0-rmse:0.06367
[900]	validation_0-rmse:0.06022
[1200]	validation_0-rmse:0.05985
[1499]	validation_0-rmse:0.0